In [ ]:
# September 7, 2023
# This notebook is the baseline for the OpenLCA workbench project. With this you can do analysis of your
# OpenLCA result files.
#
# Just run each cell of the notebook sequentially.
#

In [ ]:
import pandas as pd
from ingestion import Ingestion

## How-to configure ingestion
You need to provide an array of DataFrame creation definitions.  They will be processed in order, thus subsequent definitions may refer to DataFrames created in precedent ones.  The result of the ingestion will be a dictionary of DataFrames, each one referred to by its dataframe_name.
#### Single DataFrame creation definition
Here are the fields in a DataFrame creation definition:
* **dataframe_name** (string):
    * the identifier of the created DataFrame, i.e. `dataframes[dataframe_name]`
* **creation_method** (string):
    * two methods are possible: `"read_from_json_files"` and `"merge"`
    * these are distinctly different and the subsequent fields depend on the creation method


##### Read from JSON files (creation method)
* **input_dir** (string):
    * the directory which contains the JSON files to be processed - currently only a single directory is supported
* **file_selection_regex** (string):
    * see this __[Python regex tutorial](https://www.dataquest.io/blog/regular-expressions-data-scientists/)__ if you need explanation on working with regex
    * in the example below, the regex matches total impact files for all housing types and all farm numbers
    * regex is powerful, good to know it
* **columns** (list):
    * this is a list of column definitions for the DataFrame that is being created
    * each column definition contains:
        * **column_name** (string):
            * this will become a DataFrame column name, so follow the appropriate syntax
        * **source** (string):
            * two sources are possible: `"from_filename"` and `"from_json"`:
        * (from_filename) **spec** (string):
            * this spec is a regex, like the file_selection_regex
            * the difference is you must specify a group capture using `(` and `)`
            * a reasonable tutorial on this is __[python-regex-capturing-groups](https://pynative.com/python-regex-capturing-groups/)__
            * this column will have one value per file
            * if other columns have multiple values per file, this value will be replicated as appropriate
        * (from_json) **spec** (string):
            * this is the most complex part of the definition
            * the format of the from_json spec is based loosely on __[jmespath](https://jmespath.org/tutorial.html)__
            * but be careful, it is a simplified version of it, so the documentation is only a rough guide
            * the spec is a series of tokens separated by `.` that specifies a "JSON path"
            * the position in the series corresponds to the depth in the JSON hierarchy
            * the last token in the spec must correpond to a value or list of values
            * briefly, the possible token values (or pairs) are:
                * `<key>` - take only the sub-JSON or the value corresponding to this key
                * `[].<key>` - take the full list of sub-JSON elements or values contained in the list at this level of the JSON
                * `[n].<key>` - take the nth list element (sub-JSON or value)
                * `__keys__[].<key>` - take the full list of sub-JSON elements or values contained at this level of the JSON
                * `__keys__[n].<key>` - take the nth element (sub-JSON or value) at this level of the JSON
            * in order to really understand this, please see __[Tutorial on using the from_json spec](test_ingest.ipynb#section_id1)__

##### Merge (creation method)
* **left_dataframe** (string):
    * the name of the "left" DataFrame to merge
    * this is considered the primary DataFrame
    * it must exist in the dictionary of DataFrames already created
* **right_dataframe** (string):
    * the name of the "right" DataFrame to merge
    * this is considered the secondary DataFrame
    * it must exist in the dictionary of DataFrames already created
* **left_merge_columns** (list):
    * list of columns in the left DataFrame to merge on
* **right_merge_columns** (list):
    * list of columns in the right DataFrame to merge on

In order to really understand this, please see __[Tutorial on merging DataFrames](test_ingest.ipynb#section_id2)__

In [ ]:
# Here is an example of the configuration used for ingestion.  It takes two different sets of JSON result files, one for total impacts
# and the other for upstream impacts, ingests them into DataFrames, then merges these DataFrames into a single combined DataFrame.
#
# You will need to adjust the "input_dir" field for each of the two file types.  You may also decide to use different fields.  We need
# to discuss the syntax of field selection in person at this point, if you want to do this.
#
dataframe_creation_list = [
    {
        "dataframe_name": "total_impacts_df",
        "creation_method": "read_from_json_files",
        "input_dir": "C:\\Users\\Grass_Valley\\Documents\\Projects\\FSPL (UBC)\\GitLab\\openlca\\resources\\Individual process data sets (processed)\\",
        "file_selection_regex": "^Conventional egg production__Farm__.*__.*\.xlsx-result-total-impacts$",
        "columns": [
            {
                "column_name": "housingType",
                "source": "from_filename",
                "spec": "^Conventional egg production__Farm__(.*)__.*\.xlsx-result-total-impacts$",
            },
            {
                "column_name": "farmNumber",
                "source": "from_filename",
                "spec": "^Conventional egg production__Farm__.*__(.*)\.xlsx-result-total-impacts$"
            },
            {
                "column_name": "impactCategory",
                "source": "from_json",
                "spec": "[].impactCategory.name"
            },
            {
                "column_name": "amount",
                "source": "from_json",
                "spec": "[].amount"
            },
            {
                "column_name": "refUnit",
                "source": "from_json",
                "spec": "[].impactCategory.refUnit"
            }
        ]
    },
    {
        "dataframe_name": "upstream_impacts_df",
        "creation_method": "read_from_json_files",
        "input_dir": "C:\\Users\\Grass_Valley\\Documents\\Projects\\FSPL (UBC)\\GitLab\\openlca\\resources\\Individual process data sets (processed)\\",
        "file_selection_regex": "^Conventional egg production__Farm__(.*)__(.*)\.xlsx-result-upstream-of-impact-category$",
        "columns": [
            {
                "column_name": "housingType",
                "source": "from_filename",
                "spec": "^Conventional egg production__Farm__(.*)__.*\.xlsx-result-upstream-of-impact-category$",
            },
            {
                "column_name": "farmNumber",
                "source": "from_filename",
                "spec": "^Conventional egg production__Farm__.*__(.*)\.xlsx-result-upstream-of-impact-category$"
            },
            {
                "column_name": "impactCategory",
                "source": "from_json",
                "spec": "[].__keys__[0]"
            },
            {
                "column_name": "upstreamFlow",
                "source": "from_json",
                "spec": "[].__keys__[0].[0].upstreamTechFlows.[].techFlow.flow.name"
            },
            {
                "column_name": "upstreamFlowAmount",
                "source": "from_json",
                "spec": "[].__keys__[0].[0].upstreamTechFlows.[].result"
            }
        ]
    },
    {
        "dataframe_name": "combined_impacts_df",
        "creation_method": "merge",
        "left_dataframe": "total_impacts_df",
        "right_dataframe": "upstream_impacts_df",
        "left_merge_columns": [
            "housingType",
            "farmNumber",
            "impactCategory"
        ],
        "right_merge_columns": [
            "housingType",
            "farmNumber",
            "impactCategory"
        ]
    }
]


In [ ]:
# After running this cell, your result will be in dataframes["combined_impacts_df"].  It is a Pandas DataFrame and
# you can analyze and visualize this as you wish.  Example visualizations are given in subsequent cells.

dataframes = Ingestion.create_dataframes(dataframe_creation_list)
dataframes["combined_impacts_df"][0:20]

In [ ]:
# Sanity check on the length of the DataFrame, you should be able to predict this from your specification
# in dataframe_creation_list.
#
len(dataframes["combined_impacts_df"])

In [ ]:
# Extract the DataFrame of interest.
#
combined_impacts_df = dataframes["combined_impacts_df"].copy()
combined_impacts_df

In [ ]:
# Add an error column, equal to 0.1 * upstreamFlowAmount, for testing
#
combined_impacts_df["error"] = 0.1 * combined_impacts_df["upstreamFlowAmount"]
combined_impacts_df

### Some DataFrame manipulation for analysis
DataFrame manipulation is a bit tricky until you get used to it.  Here we will plot a stacked bar chart where:
* the housing type is A
* the impact category is Human toxicity - CML 2 baseline 2000
* the impacts are stacked by upstream flow
* there is one bar per farm
* There is a single error bar at the top of each bar representing total uncertainty

Let's look at this step by step.

#### Filter on housing type
*Note that in this example we are already filtered on housing type A because that was the only housing type represented in the input files.*

Run the cell below.

In [ ]:
housing_type = "A"
selected_impacts_df = combined_impacts_df.loc[combined_impacts_df["housingType"] == housing_type]
selected_impacts_df

#### Filter on impact category
Run the cell below.

In [ ]:
impact_category = "Human toxicity - CML 2 baseline 2000"
selected_impacts_df = selected_impacts_df.loc[selected_impacts_df["impactCategory"] == impact_category]
selected_impacts_df

#### Get a list of upstream flows
We will use this list to break the full DataFrame into multiple DataFrames, one per upstream flow.

Run the cell below.

In [ ]:
upstream_flow_list = selected_impacts_df["upstreamFlow"].unique()
upstream_flow_list

#### Get information for labelling
We are getting the ref_unit and bar tags.  In this case the bar tag is the farm number.  It is used for positioning and labelling the x-axis of the bar.

*Note that we take a copy of the DataFrame and work with the copy.  In general, Pandas does not like to read and write a DataFrame at the same time for data consistency.  So, it is good practice to take a copy.  You'll get a warning whenever Pandas thinks there is a problem.*

Run the cell below.

In [ ]:
ref_unit = selected_impacts_df["refUnit"].iloc[0]
tagged_impacts_df = selected_impacts_df.copy()
tagged_impacts_df["bar_tag"] = selected_impacts_df["farmNumber"]
print(f"ref_unit: {ref_unit}")
tagged_impacts_df

#### Split the full DataFrame by upstream flow
Run the cell below.

In [ ]:
upstream_flow_dataframes = {}
for upstream_flow in upstream_flow_list:
    upstream_flow_dataframes[upstream_flow] = tagged_impacts_df.loc[tagged_impacts_df["upstreamFlow"] == upstream_flow].reset_index()

print(f"There are now {len(upstream_flow_dataframes)} DataFrames:")
for key in upstream_flow_dataframes:
    print(f"  - {key}")

example_upstream_flow = 'Pullets-conventional'
print(" ")
print(f"For example, the {example_upstream_flow} DataFrame:")
upstream_flow_dataframes[example_upstream_flow]

### Now the actual plotting
**The following link is GOLD for examples of how to do plotting:**

>__[matplotlib gallery](https://matplotlib.org/stable/gallery/index.html)__

#### The stacked bar plot without a legend.
Loop through the upstream flows, for each:
* set the bottom of the bar segment to the top of the previous (0 for the first)
* set the top of the bar segment to the bottom plus the upstream flow amount
* add that bar segment to the plot, each bar segment added will have its own color (the legend will be dealt with later)
* keep a running total of the error
* for the last bar segment, plot the cumulative error bar

Note that this is being done for all farms simultaneously.

And, finally, do the labelling.

In [ ]:
# This is for the plot without the legend

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

cumval=None
fig, ax = plt.subplots(figsize=(12,8))

flow_count = len(upstream_flow_list)
flow_index = 0
for upstream_flow in upstream_flow_list:
    print(upstream_flow)
    upstream_flow_df = upstream_flow_dataframes[upstream_flow]
    
    if cumval is None:
        cumval = pd.DataFrame()
        cumval["bottom"] = upstream_flow_df["upstreamFlowAmount"] * 0
        cumval["error"] = upstream_flow_df["error"]
    if flow_index < flow_count - 1:
        # for all flows but the last, plot bar without error
        plt.bar(
            x=upstream_flow_df["bar_tag"], 
            height=upstream_flow_df["upstreamFlowAmount"], 
            bottom=cumval["bottom"],
        )
    else:
        # for last flow, plot bar with error
        plt.bar(
            x=upstream_flow_df["bar_tag"], 
            height=upstream_flow_df["upstreamFlowAmount"], 
            bottom=cumval["bottom"], 
            yerr=cumval["error"], align='center', alpha=0.5, ecolor='gray', capsize=4
        )

    cumval["bottom"] = cumval["bottom"].add(upstream_flow_df["upstreamFlowAmount"])
    cumval["error"] = cumval["error"].add(upstream_flow_df["error"])
    flow_index += 1

_ = plt.xticks(rotation=30)
ax.set_xlabel("Farm number")
ax.set_ylabel(f"{impact_category} ({ref_unit})")
ax.set_title(f"Upstream impacts for housing type {housing_type} by farm number")

plt.show()

#### The legend.
This is done in a manner similar to the real plot.  It is just that the bar segments all have 0 height.  Also, the legend is exported as a JPEG to file.

In [ ]:
# This is for a separate legend

import pandas as pd
import matplotlib.pyplot as plt

def export_legend(legend, filename="legend.png"):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi="figure", bbox_inches=bbox)

cumval=None
fig = plt.figure(figsize=(12,8))
for upstream_flow in upstream_flow_list:
    upstream_flow_df = upstream_flow_dataframes[upstream_flow]
    if cumval is None:
        cumval = pd.DataFrame()
        cumval["bottom"] = upstream_flow_df["upstreamFlowAmount"] * 0
    plt.bar(x=upstream_flow_df["bar_tag"], height=cumval["bottom"], bottom=cumval["bottom"], label=upstream_flow)

_ = plt.xticks(rotation=30)
legend = plt.legend(fontsize=18)

legend_filename="legend.png"
export_legend(legend, filename=legend_filename)
print(f"\nThe legend without the bounding box is in the file: {legend_filename}")
plt.show()